N12-Crawling

In [ ]:
import numpy
import pandas as pd
import requests
from bs4 import BeautifulSoup as bu

n_pages = 4 # 2046
Time_Date = list()
Title = list()
Wrote = list()
Link = list()


for n in range(1, n_pages + 1):
  print(n)
  url = "https://www.mako.co.il/news-military?page=" + str(n)
  response = requests.get(url)
  soup = bu(response.content, "html.parser")
  print(response)
  print(soup)
  uls = soup("ul", attrs={"class": "more-items"})

  for ul in uls:
    for li in ul.findAll('li'):
      ah = li.p.a
      Title.append(ah.text.replace("\n", " "))
      Link.append("mako.co.il" + ah.attrs["href"])
      spn = li.find_all("span")
      try:
        Wrote.append(spn[0].text.replace("\n", " "))
      except IndexError:
        Wrote.append(numpy.NaN)
      try:
        Time_Date.append(spn[1].text)
      except IndexError:
        Time_Date.append(numpy.NaN)
  print(len(Time_Date), ",", len(Title), ",", len(Wrote), ",", len(Link))

df = pd.DataFrame({"Time_Date": Time_Date, "Title": Title, "Wrote": Wrote, "Link": Link})
df.to_csv('N12-Crawling.csv')




N12_DataProcessing

In [ ]:
import pandas as pd
import matplotlib as plt


df1 = pd.read_csv('**********/N12-Crawling.csv', lineterminator='\n').copy()

df1['Date'] = pd.to_datetime(df1['Time_Date'], dayfirst=True)
df1['Year'] = pd.DatetimeIndex(df1['Date']).year
df1['Week_Year'] = df1['Date'].dt.strftime("%G %V")
df.sort_values(by = ['Date'], inplace = True)

Monthdf = df1['Week_Year'].value_counts(sort=False)
Monthdf.sort_index(inplace=True)


df.drop(df.index[df['Year'] <= 2010], inplace=True)
df.dropna(subset = ['Year'], inplace=True)

df.to_csv('N12_DaPro.csv')

Ta_125_Selenium

In [ ]:
import pandas as pd
import time
import sys
from selenium import webdriver
from selenium.webdriver.common.keys import Keys
from selenium.webdriver.support import expected_conditions as EC
from selenium.webdriver.common.by import By
from selenium.webdriver.support.wait import WebDriverWait
sys.path.insert(0,'/usr/lib/chromium-browser/chromedriver')

driver = webdriver.Chrome(r"*********/chromedriver")
driver.get("https://il.investing.com/indices/ta100-historical-data")
WebDriverWait(driver, 10).until(EC.element_to_be_clickable((By.ID, 'data_interval'))).click()
WebDriverWait(driver, 10).until(EC.element_to_be_clickable((By.ID, 'widgetFieldDateRange'))).click()
date = WebDriverWait(driver, 10).until(EC.element_to_be_clickable((By.ID, 'startDate')))
date.clear()
date.send_keys("01/01/2011",Keys.ENTER)
time.sleep(3)
df_TA125 = pd.read_html(driver.page_source)[0]

time.sleep(3)
df_TA125.to_csv('TA-125.csv')


ILS_USD_Selenium

In [ ]:
import pandas as pd
import time
from selenium import webdriver
from selenium.webdriver.common.keys import Keys
from selenium.webdriver.support import expected_conditions as EC
from selenium.webdriver.common.by import By
from selenium.webdriver.support.wait import WebDriverWait

driver = webdriver.Chrome(r"************/chromedriver")


driver.get("https://il.investing.com/currencies/usd-ils-historical-data")
WebDriverWait(driver, 10).until(EC.element_to_be_clickable((By.ID, 'widgetFieldDateRange'))).click()
date = WebDriverWait(driver, 10).until(EC.element_to_be_clickable((By.ID, 'startDate')))
date.clear()
date.send_keys("01/01/2011",Keys.ENTER)
time.sleep(3)
df_us = pd.read_html(driver.page_source)[0]

time.sleep(3)
df_us.to_csv('ILS-USD.csv')

Wikipedia Crawling

In [ ]:
import numpy
import pandas as pd
import requests
import lxml.html as lh


url = "https://he.wikipedia.org/wiki/מבצעי_צבא_הגנה_לישראל"
page = requests.get(url)
doc = lh.fromstring(page.content)
tr_elements = doc.xpath('//tbody')[15]


NameOper = list()
PurpOper = list()
DateOper = list()
for t in tr_elements:
    NameOper.append(t[0].text_content().replace("\n", ""))
    PurpOper.append(t[1].text_content().replace("\n", ""))
    DateOper.append(t[2].text_content().replace("\n", ""))

df = pd.DataFrame({'NameOper':NameOper, 'PurpOper':PurpOper, 'DateOper':DateOper})
df.to_csv('HiOperIDF.csv')

ILS-USD_DataProcessing

In [ ]:
import pandas as pd

df_USD = pd.read_csv('*************/ILS-USD.csv', lineterminator='\n').copy()

df_USD = df_USD.rename({'תאריך': 'Date', 'שער': 'Rate', 'פתיחה':'Open', 'גבוה':'High', 'נמוך':'Low', 'שינוי %':'Change %'}, axis=1).copy()

df_USD.drop(columns=['Unnamed: 0'], inplace=True)

df_USD['Date'] = pd.to_datetime(df_USD['Date'], dayfirst=True)
df_USD['Year'] = pd.DatetimeIndex(df_USD['Date']).year
df_USD['Week_Year'] = df_USD['Date'].dt.strftime("%G %V")
df_USD['Change %'] = df_USD['Change %'].str.rstrip('%').astype('float')

df_USD.to_csv('DaPr_USD.csv')

EDA-USD_Rate

In [ ]:

import pandas as pd
import matplotlib.pyplot as plt
import numpy as np
import seaborn as sns

N12_df = pd.read_csv('************/N12_DaPro.csv', lineterminator='\n').copy()
N12_df['Date'] = pd.to_datetime(N12_df['Time_Date'], dayfirst=True) # Not sure why this is needed again
N12_df['Week_Year'] = N12_df['Date'].dt.strftime("%G %V")

N12_daily = N12_df['Date'].value_counts(sort=False).rename("Daily articles").copy()
N12_daily.sort_index(inplace=True)
N12_weekly = N12_df['Week_Year'].value_counts(sort=False).rename("Weekly articles").copy()


USD_df = pd.read_csv('***************/ILS-USD.csv', lineterminator='\n').copy()
USD_df = USD_df.rename({'תאריך': 'Date', 'שער': 'Rate', 'פתיחה':'Open', 'גבוה':'High', 'נמוך':'Low', 'שינוי %':'Change %'}, axis=1).copy()
USD_df.drop(columns=['Unnamed: 0'], inplace=True)
USD_df['Date'] = pd.to_datetime(USD_df['Date'], dayfirst=True)
USD_df['Year'] = pd.DatetimeIndex(USD_df['Date']).year
USD_df['Week_Year'] = USD_df['Date'].dt.strftime("%G %V")
USD_df.set_index('Date', inplace=True)
USD_df['Change %'] = USD_df['Change %'].str.rstrip('%').astype('float')
USD_df.drop(USD_df.index[USD_df['Year'] <= 2010], inplace=True)

USD_week = USD_df.groupby(['Week_Year']).max()
USD_week = pd.concat([USD_week, USD_week['Rate'].pct_change().rename("Interweek change").shift(periods=-1)], axis=1).copy()

Weekly_unified_df = pd.concat([USD_week, N12_weekly], axis=1).sort_index()
fig, ax = plt.subplots()
Weekly_unified_df['Weekly articles'].plot(ax=ax, style='b-', figsize=(15, 10))
Weekly_unified_df['High'].plot(ax=ax, style='r-', secondary_y=True)
ax.legend([ax.get_lines()[0], ax.right_ax.get_lines()[0]],\
           ['Weekly articles','USD rate'], bbox_to_anchor=(0.9, 0.9))
fig.show()

print("Average ILS/USD rate:")
print(USD_df['Change %'].mean())

unified_df = pd.concat([USD_df, N12_daily], axis=1).sort_index().copy()
unified_df.to_csv("alldataframe.csv")

BigArticlesTLV_df = unified_df[unified_df['Daily articles'] >= 20].copy()
print ("Average TLV 125 change at Insecure Time:")
print(BigArticlesTLV_df['Change %'].mean())

EDA_NotDone_OnlyforTestNOw

In [ ]:
from numpy.core.numeric import NaN
import pandas as pd
import matplotlib.pyplot as plt
import numpy as np


n12 = pd.read_csv('*************/N12_DaPro.csv', lineterminator='\n').copy()
df_USD = pd.read_csv('***********/DaPr_USD.csv', lineterminator='\n').copy()


#print(n12['Year'].value_counts(sort=False))
# # Week_Num_bins = [datetime.strptime("0000 0", "%Y %V"), datetime.strptime("2008 51","%Y %V")  ,datetime.strptime("2009 3","%Y %V"), datetime.strptime("2012 45","%Y %V"), datetime.strptime("2012 47","%Y %V"), datetime.strptime("2014 22","%Y %V"), datetime.strptime("2014 35","%Y %V"), datetime.strptime("2021 17","%Y %V"), datetime.strptime("2021 20","%Y %V")]

bins = [0, 2011, 2012, 2013, 2014, 2020, 2021] #2008-2009, 2012, 2014, 2021
names = [np.NaN, 'Operation Pillar of Defense', np.NaN, 'Operation Protective Edge', np.NaN, 'Israel–Palestine crisis']


n12['TimeOfWar'] = pd.cut(n12.Year, bins, labels=names, ordered=False)
df_USD['TimeOfWar'] = pd.cut(df_USD.Year, bins, labels=names, ordered=False)

print(n12)

# df_articles = n12[n12['TimeOfWar'] != np.NaN].copy()
# # df_articles = df_articles.dropna(how='any')
# plt.figure()
# # df_articles = df_articles['Week_Year'].groupby(df_articles['TimeOfWar'])
# df_articles['Week_Year'].value_counts(sort=False).plot.bar()
# plt.show()
# print(df_articles)


print(n12['Year'].value_counts(sort=False))
plt.figure()
n12['Year'].value_counts(sort=False).plot.bar(color=['black', 'red', 'black', 'red', 'black', 'black', 'black', 'black', 'black', 'black', 'red'])

plt.title('כמות מאמרים בכל שנה'[::-1])

plt.show()

print(n12.info())

plt.figure()
n12['Year'].groupby(n12['TimeOfWar']).value_counts(sort=False).plot.bar(color=['blue','cyan','blue'])
# plt.savefig('/content/drive/MyDrive/visualization/?.png')
plt.title('כמות מאמרים בשנים של מבצעים משמעותיים'[::-1])
plt.show()

# ######################################################
df_USD['Year'].groupby(df_USD['TimeOfWar']).value_counts(sort=False)
# import seaborn as sns

# # left
# sns.pairplot(n12, kind="scatter", hue="TimeOfWar", markers=["o", "s", "D"], palette="Set2")
# plt.show()


import seaborn as sns

# left
sns.pairplot(df_USD, kind="scatter", hue="TimeOfWar", markers=["o", "s", "D"], palette="Set2")
plt.show()

# Year_of_war = [2008, 2009, 2012, 2014, 2021]
# Name_of_war  = ['Operation Cast Lead', 'Operation Cast Lead', 'Operation Pillar of Defense', 'Operation Protective Edge', 'Israel–Palestine crisis']

# df_war = {'Year':Year_of_war, 'Name':Name_of_war}
#Amount_of_articles =
# n12Year = [n12['Year'].value_counts(sort=False)[0]]
# n12Amount = [n12['Year'].value_counts(sort=False)[1]]
# df_articles = {'Year':n12Year, 'AmountOfArticles':n12Amount}

EDA_Ta-125_NotDone

In [ ]:
import pandas as pd
import matplotlib.pyplot as plt
import numpy as np
import seaborn as sns
import datetime
import numpy

#seaborn -> sns.plot -> LM.plot הכי יתאים?

N12_df = pd.read_csv('*************/N12_DaPro.csv', lineterminator='\n').copy()
N12_df['Date'] = pd.to_datetime(N12_df['Time_Date'], dayfirst=True) # Not sure why this is needed again
N12_df['Week_Year'] = N12_df['Date'].dt.strftime("%G %V")

N12_daily = N12_df['Time_Date'].value_counts(sort=False).rename("Daily articles").copy()
N12_daily.sort_index(inplace=True)
N12_weekly = N12_df['Week_Year'].value_counts(sort=False).rename("Weekly articles").copy()

USD_df = pd.read_csv('***************/DaPr_USD.csv', lineterminator='\n').copy()

# USD_df['Date'] = pd.to_datetime(USD_df['Date'], dayfirst=True)
# USD_df['Year'] = pd.DatetimeIndex(USD_df['Date']).year
# USD_df['Week_Year'] = USD_df['Date'].dt.strftime("%G %V")
# USD_df.set_index('Date', inplace=True)
# USD_df['Change %'] = USD_df['Change %'].str.rstrip('%').astype('float')

# USD_df.drop(USD_df.index[USD_df['Year'] <= 2010], inplace=True)
USD_df.drop(columns=['Unnamed: 0'], inplace=True)
print(USD_df.info())
USD_df.set_index('Date', inplace=True)

USD_week = USD_df.groupby(['Week_Year']).max()
USD_week = pd.concat([USD_week, USD_week['Rate'].pct_change().rename("Interweek change").shift(periods=-1)], axis=1).copy()
print(USD_week.info())

Weekly_unified_df = pd.concat([USD_week, N12_weekly], axis=1).sort_index()

Weekly_unified_df.to_csv('/content/drive/MyDrive/datasets/TESTWeekly_unified_df.csv')

########################################### "מנקה את הרעשים"
Weekly_unified_df = Weekly_unified_df.rolling(10).sum()
######################## עושה ממוצע על כמה נקודות, כלומר עושה ממוצע בין כל 10 שבועות
######################## לסיכום- מחליק את הגרף על תקופה של 10 שבועות
fig, ax = plt.subplots()
Weekly_unified_df['Weekly articles'].plot(ax=ax, style='b-', figsize=(15, 10))
Weekly_unified_df['High'].plot(ax=ax, style='r-', secondary_y=True)
ax.legend([ax.get_lines()[0], ax.right_ax.get_lines()[0]],\
           ['Weekly articles','USD rate'], bbox_to_anchor=(0.9, 0.9))

################################################################
#operations:

# #(2008-2009) ###############הורדנו
# start_cast_lead = 50 #date(2008, 12, 27)27.12.2008
# end_cast_lead = 100 #date(2009, 1, 18)18.1.2009
# plt.axvspan(start_cast_lead, end_cast_lead, facecolor='y', alpha=0.3)
# ######################################################


######################## חישוב
# year1, weekofyear1
# year2, weekofyear2
# (year2-year1)*52+(weekofyear2-weekofyear1)
#########################

####מלחמה ראשונה מתחילת 2011
plt.axvspan(98, 99, facecolor='g', alpha=0.9) #######הסבר בקובץ שלי "המשך הסבר על הסימונים בגרפים", חשוב למצגת
plt.text(18, 32, 'Pilar Of Defense\n'+'עמוד ענן'[::-1]+'\nVictory!') #מ98 לסמן בגרף

# plt.axvspan(start_pilar_of_defense,end_pilar_of_defense)
# plt.text(start_pilar_of_defense, 200, 'Pilar Of Defense')

####מלחמה שניה מתחילת 2011
plt.axvspan(179, 186, facecolor='y', alpha=0.5)  #מ179 לסמן בגרף
plt.text(100, 32, 'Protective Edge 2014\n'+'מבצע צוק איתן'[::-1]+'\nBoth sides claim victory') #######הסבר בקובץ שלי "המשך הסבר על הסימונים בגרפים", חשוב למצגת

# #7 weeks
# plt.axvspan(start_protective_edge_2014,end_protective_edge_2014)
# plt.text(start_protective_edge_2014, 170, 'Protective Edge 2014')


####מלחמה שלישית מתחילת 2011
plt.axvspan(538, 540.1428571428571, facecolor='m', alpha=0.6)
# plt.axvspan(start_2021_Israel_Palestine_crisis, end_2021_Israel_Palestine_crisis) להמחשה
plt.text(428, 32, '2021 Israel-Palestine Crisis\n'+'מבצע שומר החומות'[::-1]+'\nBoth sides claim victory') #קצת לפני איפה שנסמן

################################################################################################ אם טוב להמשיך
# plt.axvspan(190, 250, facecolor='g', alpha=0.3)
# plt.savefig('/content/drive/MyDrive/visualization/Weeklyarticles_high_linear-regression.jpg') #png בלי רקע
fig.show()

######הוספתי
# plt.fig()
# ax1 = Weekly_unified_df.plot.scatter(x='Weekly articles', y='High', c='Red')
# fig.show()
# ##########################



#לנסות לתקן אם אפשר


# ##########################
print("Weekly articles_High Correlation:")
print(Weekly_unified_df['Weekly articles'].corr(Weekly_unified_df['High']))
##### ערך הקורלציה הינו
##### 0.359
##### Weekly_unified_df['High'] מה שאומר שיש קורלציה חיובית די נמוכה בין כמות הכתבות פר שבוע לבין ערך
##### זה אומר שכשמספר הכתבות גבוה אז ערך המטבע עולה
####### זה לא מסתדר עם מה שרצינו להראות?
print('value of x(Weekly articles)', Weekly_unified_df['Weekly articles']) #לשם בדיקה
print('value type of x(Weekly articles)', type(Weekly_unified_df['Weekly articles'])) #לשם בדיקה
#################################################################

####################### נבדוק קשר לינארי בין המשתנים
x = Weekly_unified_df['High']
y = Weekly_unified_df['Weekly articles']
# number of observations/points
n = np.size(x)

# mean of x and y vector
m_x = np.mean(Weekly_unified_df['High'])
m_y = np.mean(Weekly_unified_df['Weekly articles'])

# calculating cross-deviation and deviation about x
SS_xy = np.sum(y*x) - n*m_y*m_x
SS_xx = np.sum(x*x) - n*m_x*m_x

# calculating regression coefficients
b_1 = SS_xy / SS_xx
b_0 = m_y - b_1*m_x
b = b_0, b_1

print("Estimated coefficients:\nb_0 = {}  \
          \nb_1 = {}".format(b[0], b[1]))

###################### plotting regression line

# plotting the actual points as scatter plot
plt.figure()
plt.scatter(x, y, color = "m",
               marker = "o", s = 30)

# predicted response vector
y_pred = b[0] + b[1]*x
# y_pred = -2600 + 85*x
# plotting the regression line
plt.plot(x, y_pred, color = "g")

# putting labels
plt.xlabel('Weekly articles')
plt.ylabel('High')
# plt.savefig('/content/drive/MyDrive/visualization/Weeklyarticles_high_linear-regression.jpg') #png בלי רקע
# function to show plot
fig.show()
#########################################################

 #LOWאז נבדוק איזה קשר לינארי קיים בין כמות כתבות ביטחוניות לערך


####################### נבדוק קשר לינארי בין המשתנים
x = Weekly_unified_df['Low']
y = Weekly_unified_df['Weekly articles']
# number of observations/points
n = np.size(x)

# mean of x and y vector
m_x = np.mean(Weekly_unified_df['Low'])
m_y = np.mean(Weekly_unified_df['Weekly articles'])

# calculating cross-deviation and deviation about x
SS_xy = np.sum(y*x) - n*m_y*m_x
SS_xx = np.sum(x*x) - n*m_x*m_x

# calculating regression coefficients
b_1 = SS_xy / SS_xx
b_0 = m_y - b_1*m_x
b = b_0, b_1

print("Estimated coefficients:\nb_0 = {}  \
          \nb_1 = {}".format(b[0], b[1]))

###################### plotting regression line

# plotting the actual points as scatter plot
plt.figure()
plt.scatter(x, y, color = "g",
               marker = "o", s = 30)

# predicted response vector
y_pred = b[0] + b[1]*x
# y_pred = -2600 + 85*x
# plotting the regression line
plt.plot(x, y_pred, color = "r")

# putting labels
plt.xlabel('Weekly articles')
plt.ylabel('Low')
# plt.savefig('/content/drive/MyDrive/visualization/Weeklyarticles_Low_linear-regression.jpg') #png בלי רקע
# function to show plot
fig.show()
########################################################
from scipy.stats import chi2_contingency
ct = pd.crosstab(Weekly_unified_df['Interweek change'],Weekly_unified_df['Weekly articles'] )
print('chi2_contingency:', chi2_contingency(ct))
print('chi2_contingency: \np = chi2_contingency:p-value = 0.2817932103859786 -> p>0.05, השערת הבסיס אינה נדחית')
print('ואולי אכן יש קשר בין כמות כתבות ביטחוניות שבועיות המעידות על משבר ביטחוני לבין שינוי בערך המטבע')
############################ נבדוק קשר לינארי בין המשתנים
x = Weekly_unified_df['Interweek change']
y = Weekly_unified_df['Weekly articles']
# number of observations/points
n = np.size(x)

# mean of x and y vector
m_x = np.mean(Weekly_unified_df['Interweek change'])
m_y = np.mean(Weekly_unified_df['Weekly articles'])

# calculating cross-deviation and deviation about x
SS_xy = np.sum(y*x) - n*m_y*m_x
SS_xx = np.sum(x*x) - n*m_x*m_x

# calculating regression coefficients
b_1 = SS_xy / SS_xx
b_0 = m_y - b_1*m_x
b = b_0, b_1

print("Estimated coefficients:\nb_0 = {}  \
          \nb_1 = {}".format(b[0], b[1]))

###################### plotting regression line

# plotting the actual points as scatter plot
plt.figure()
plt.scatter(x, y, color = 'b',
               marker = 'o', s = 30)

# predicted response vector
y_pred = b[0] + b[1]*x
# y_pred = -2600 + 85*x
# plotting the regression line
plt.plot(x, y_pred, color = "r")

# putting labels
plt.xlabel('Weekly articles')
plt.ylabel('Interweek change')

# function to show plot
# plt.savefig('/content/drive/MyDrive/visualization/Weeklyarticles_Interweekchange_linear-regression.jpg') #png בלי רקע
fig.show()
print("Correlation:")
print(Weekly_unified_df['Weekly articles'].corr(Weekly_unified_df['Interweek change']))
##############################################################################

#####################################################
#המשכתי את מה שנתן עשה, לא יודעת למה רלוונטי
print ("Average ILS/USD rate:")
print(USD_df['Change %'].mean())

unified_df = pd.concat([USD_df, N12_daily], axis=1).sort_index().copy()

BigArticlesTLV_df = unified_df[unified_df['Daily articles'] >= 20].copy()

print ("Average TLV 125 change at Insecure Time:")
avg = BigArticlesTLV_df['Change %'].mean()
print(avg)

print ("Standard deviation TLV 125 change at Insecure Time:")
std =  BigArticlesTLV_df['Change %'].std()
print(std)

# plt.text(x, y, r'Average is {avg}')
# plt.text(x, y, r'Standard deviation is {std}') # לערכים של ההיסטוגרמה עליה אנו רוצים לצייר אותם
# #להוסיף את הכיתוב בעזרת הפונקציה plt.text
# #X ו Y בהתאם לגרף במקומות של
plt.figure()
plt.text(-1.5, 25, ('Average is {:2f}').format(avg)) #x וy צריך להתאים את
plt.text(-1.5, 22, ('Standard deviation is {:2f}').format(std)) # לערכים של ההיסטוגרמה עליה אנו רוצים לצייר אותם
plt.hist(BigArticlesTLV_df['Change %'])

plt.title('אחוז השינוי בדולר כאשר יש 02 ומעלה מאמרים ביום'[::-1])
# plt.xlabel('אחוז השינוי'[::-1])
# plt.ylabel('כמות כתבות ליום'[::-1])
# plt.savefig('/content/drive/MyDrive/visualization/20OrMore.jpg') #png בלי רקע
fig.show()


##########################לשאול את נתן איך
# between0to5 = BigArticlesTLV_df[BigArticlesTLV_df['Daily articles'] <=5].copy()
# avg = between0to5['Change %'].mean()
# std =  between0to5['Change %'].std()
# plt.figure()
# plt.text(-1.5, 25, ('Average is {:2f}').format(avg)) #x וy צריך להתאים את
# plt.text(-1.5, 22, ('Standard deviation is {:2f}').format(std)) # לערכים של ההיסטוגרמה עליה אנו רוצים לצייר אותם
# plt.hist(between0to5['Change %'])

# plt.title('אחוז השינוי בדולר כאשר יש בין 0 ל5 מאמרים ביום'[::-1])
# plt.xlabel('אחוז השינוי'[::-1])
# plt.ylabel('כמות כתבות ליום'[::-1])
# plt.savefig('C:/Users/chen/Desktop/PythonProject/between0to5.jpg') #png בלי רקע
# fig.show()

####################################################

EDA_Unified_BigDataFrame

In [ ]:
import pandas as pd
import seaborn as sns

df = pd.read_csv('************************/TESTWeekly_unified_df.csv', lineterminator='\n').copy()
# Density
# sns.pairplot(df, diag_kind="kde")

# Histogram
# sns.pairplot(df, diag_kind="hist")

# You can custom it as a density plot or histogram so see the related sections
sns.pairplot(df, diag_kind="kde", diag_kws=dict(shade=True, bw_adjust=.05, vertical=False) )

plt.show()


bins = [0, 2011, 2012, 2013, 2014, 2020, 2021] #2008-2009, 2012, 2014, 2021
names = [np.NaN, 'Operation Pillar of Defense', np.NaN, 'Operation Protective Edge', np.NaN, 'Israel–Palestine crisis']

df['TimeOfWar'] = pd.cut(df.Year, bins, labels=names, ordered=False)

# df['Year'].groupby(df['TimeOfWar']).value_counts(sort=False)
# left
sns.pairplot(df, kind="scatter", hue="TimeOfWar", markers=["o", "s", "D"], palette="Set2")
# plt.savefig('/content/drive/MyDrive/visualization/ערךהשקל-דולרבזמנימלחמהוכתבות.png')
plt.show()


df.dropna(how='any', inplace=True)
# # set figure size
plt.figure(figsize=(20,10))

# plot polar axis
ax = plt.subplot(111, polar=True)

# remove grid
plt.axis('off')

# Set the coordinates limits
upperLimit = 100
lowerLimit = 30

# Compute max and min in the dataset
max = df['Interweek change'].max()

# Let's compute heights: they are a conversion of each item value in those new coordinates
# In our example, 0 in the dataset will be converted to the lowerLimit (10)
# The maximum will be converted to the upperLimit (100)
slope = (max - lowerLimit) / max
heights = slope * df['Interweek change'] + lowerLimit

# Compute the width of each bar. In total we have 2*Pi = 360°
width = 2*np.pi / len(df.index)

# Compute the angle each bar is centered on:
indexes = list(range(1, len(df.index)+1))
angles = [element * width for element in indexes]
angles

# initialize the figure
plt.figure(figsize=(20,10))
ax = plt.subplot(111, polar=True)
plt.axis('off')

# Draw bars
bars = ax.bar(
    x=angles,
    height=heights,
    width=width,
    bottom=lowerLimit,
    linewidth=2,
    edgecolor="white",
    color=['coral', 'blueviolet', 'chocolate' ],#'"#61a4b2",
)

# little space between the bar and the label
labelPadding = 4

# Add labels
for bar, angle, height, label in zip(bars,angles, heights, df["TimeOfWar"]):

    # Labels are rotated. Rotation must be specified in degrees :(
    rotation = np.rad2deg(angle)

    # Flip some labels upside down
    alignment = ""
    if angle >= np.pi/2 and angle < 3*np.pi/2:
        alignment = "right"
        rotation = rotation + 180
    else:
        alignment = "left"

    # Finally add the labels
    ax.text(
        x=angle,
        y=lowerLimit + bar.get_height() + labelPadding,
        s=label,
        ha=alignment,
        va='center',
        rotation=rotation,
        rotation_mode="anchor")
###########################################################################

EDA_Opertition_Year&Mean

In [ ]:
import pandas as pd
import seaborn as sns
import matplotlib.pyplot as plt
import numpy as np

wars_d = {'Year':[2011, 2012, 2013, 2014, 2018, 2019, 2021], 'OperationsAmount':[1,1,1,3,1,3,1]}
wars_df= pd.DataFrame(wars_d)
print(wars_df)
# wars_df['OperationsAmount'].groupby(wars_df['Year'])

# create data: an array of values
size_of_groups=[1,1,1,3,0,0,0,1,3,1]
values=[1,1,1,3,1,3,1]
names='2011', '2012', '2013', '2014', '2018', '2019', '2021',
# Create a pieplot
plt.pie(size_of_groups)
plt.pie(values, labels=names, labeldistance=1.15)
plt.title('  Operations per year 2011-2021',   fontweight ="bold")
# plt.savefig('/content/drive/MyDrive/visualization/Operationsperyear2011-2021.png'
plt.show()
###########################################################################
print(wars_df.describe())

Uni_df = pd.read_csv('****************/TESTWeekly_unified_df.csv', lineterminator='\n').copy()

Uni_df['Weekly articles'].groupby(Uni_df['Year'])
# Uni_df.dropna(how='any')#YEARLY_ARTICLES
# size_of_groups= [for i in Uni_df['Weekly articles']]
# # Create a pieplot
# plt.pie(size_of_groups)
# # names='2011', '2012', '2013', '2014', '2018', '2019', '2021',
# # # plt.pie(Uni_df['Weekly articles'], labels=names, labeldistance=1.15);
# # plt.show()
###########################################################################
bins = [0, 2010, 2011, 2012, 2013, 2014, 2017, 2018, 2019, 2020, 2021] #2008-2009, 2012, 2014, 2021
names = [np.NaN, '2011 Single Operation', '2012 Single Operation', '2013 Single Operation', '2014 Three Operations', np.NaN, '2018 Single Operation', '2019 Three Operations', np.NaN, '2021 Single Operation']
Uni_df['OperationsAmountPerYear'] = pd.cut(Uni_df.Year, bins, labels=names, ordered=False)

Uni_df['OperationsAmountPerYear']= Uni_df['OperationsAmountPerYear'].cat.add_categories('Zero Operations')
Uni_df['OperationsAmountPerYear'].fillna('Zero Operations', inplace=True)
# Uni_df['OperationsAmountPerYear'] = Uni_df['OperationsAmountPerYear'].fillna('No Operations')
# d = Uni_df['Year', 'Interweek change', 'Weekly articles'].groupby(Uni_df['OperationsAmountPerYear']).value_counts(sort=False)#.plot.bar()#color=['blue','cyan','blue'])
# d = Uni_df['Year'].groupby(Uni_df['OperationsAmountPerYear'])#.value_counts(sort=False)#.plot.bar()#color=['blue','cyan','blue'])
grouped_multiple = Uni_df.groupby(['OperationsAmountPerYear']).agg({'Weekly articles': ['mean', 'min', 'max'], 'Interweek change': ['mean', 'min', 'max']})
grouped_multiple.columns = ['Weekly articles_mean', 'Weekly articles_min', 'Weekly articles_max', 'Interweek change_mean', 'Interweek change_min', 'Interweek change_max']
grouped_multiple = grouped_multiple.reset_index()
print(grouped_multiple)
grouped_multiple.to_csv('grouped_multiple_Uni_war_df.csv')
# grouped_multiple.plot.bar()
# plt.show()
# res=gb_sales_demo.reset_index()
d_mean = grouped_multiple[["OperationsAmountPerYear",'Weekly articles_mean','Interweek change_mean']].copy()
d_mean.to_csv('Mean_grouped_multiple_Uni_war_df.csv')
# d_mean = Uni_df.groupby(['OperationsAmountPerYear']).agg({'Weekly articles': ['mean'], 'Interweek change': ['mean']})
# res=d_mean.reset_index()
print(d_mean)

# res = d_mean.reset_index()
# print(res)
res_wide=d_mean.melt(id_vars="OperationsAmountPerYear")
print(res_wide)

plt.figure(figsize=(20,16))
sns.set_theme(style="whitegrid")
sns.barplot(x='OperationsAmountPerYear', y="value",data=res_wide, hue="variable",palette='magma')
# sns.catplot(x="OperationsAmountPerYear", kind="count", palette="ch:.25")
# Uni_df[['Interweek change', 'Weekly articles']].groupby(Uni_df['OperationsAmountPerYear']).value_counts(sort=False).plot.bar()#color=['blue','cyan','blue'])
# d['Interweek change', 'Weekly articles']


plt.legend(prop ={'size': 10})

plt.title('מחולק לפי שנים וכמות אירועים ביטחוניים \nממוצע כמות כתבות שבועיות וממוצע אחוז שינוי שבועי בשקל-דולר\n\n'[::-1],
          fontweight ="bold")

plt.show()
d_mean['OperationsAmount'] = [1,1,1,3,1,3,1,0]
d_mean['Year'] = [2011, 2012, 2013, 2014, 2018, 2019, 2021, np.NAN]
grid = sns.JointGrid(x='OperationsAmount', y='Weekly articles_mean', data=d_mean)

g = grid.plot_joint(sns.scatterplot, hue='Interweek change_mean', data=d_mean)
sns.kdeplot(d_mean.loc[d_mean['Interweek change_mean']>0, 'OperationsAmount'], ax=g.ax_marg_x, legend=False)
sns.kdeplot(d_mean.loc[d_mean['Interweek change_mean']>0, 'OperationsAmount'], ax=g.ax_marg_x, legend=False)
sns.kdeplot(d_mean.loc[d_mean['Interweek change_mean']<0, 'Weekly articles_mean'], ax=g.ax_marg_y, vertical=True, legend=False)
sns.kdeplot(d_mean.loc[d_mean['Interweek change_mean']<0, 'Weekly articles_mean'], ax=g.ax_marg_y, vertical=True, legend=False)

print(d_mean)
#################################????
pd.crosstab(d_mean['Weekly articles_mean'],d_mean.OperationsAmount).plot(kind='bar')
print('\n\n\n')
pd.crosstab(d_mean['Interweek change_mean'],d_mean.OperationsAmount).plot(kind='bar')
pd.crosstab(d_mean.OperationsAmount,d_mean['Interweek change_mean']).plot(kind='line')
################################???


# Use the 'hue' argument to provide a factor variable
sns.lmplot( x="Weekly articles_mean", y="Interweek change_mean", data=d_mean, fit_reg=False, hue='OperationsAmount', legend=False)

# Move the legend to an empty part of the plot
plt.legend(loc='lower right')

plt.show()
# Use the 'hue' argument to provide a factor variable
sns.lmplot( x="Weekly articles_mean", y="Interweek change_mean", data=d_mean, fit_reg=True, hue='OperationsAmount', legend=False)

# Move the legend to an empty part of the plot
plt.legend(loc='lower right')
plt.title('משמע בין השנים 2014-2019 \nניתן לראות את הקשר בין עלייה בכמות כתבות ביטחוניות ועליה בערך השקל-דולר כאשר אירעו 3 אירועים ביטחוניים בשנה\n\n'[::-1],
          fontweight ="bold")
plt.show()



# Use the 'hue' argument to provide a factor variable
sns.lmplot( x='Year', y="Interweek change_mean", data=d_mean, fit_reg=False, hue='OperationsAmount', legend=False)

# # Move the legend to an empty part of the plot
plt.legend(loc='lower right')

plt.show()


# Use the 'hue' argument to provide a factor variable
sns.lmplot( x='Year', y="Interweek change_mean", data=d_mean, fit_reg=True, hue='OperationsAmount', legend=False)

# # Move the legend to an empty part of the plot
plt.legend(loc='lower right')

plt.show()
# # Use the 'hue' argument to provide a factor variable
# sns.lmplot( x="Weekly articles_mean", y="Interweek change_mean", data=d_mean, fit_reg=True, hue='OperationsAmount', legend=False)

# # Move the legend to an empty part of the plot
# plt.legend(loc='lower right')

# plt.show()

# Use the 'hue' argument to provide a factor variable
sns.lmplot( x='OperationsAmount', y="Interweek change_mean", data=d_mean, fit_reg=True, legend=False)

# # Move the legend to an empty part of the plot
plt.legend(loc='lower right')

plt.show()

# Use the 'hue' argument to provide a factor variable
sns.lmplot( x='OperationsAmount', y="Interweek change_mean", data=d_mean, fit_reg=True,hue='Year', legend=False)

# # Move the legend to an empty part of the plot
plt.legend(loc='lower right')

plt.show()